In [203]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import yfinance as yf
import pygad
import random

from deap import algorithms, base, creator, tools

In [209]:
stocks = ['ITUB4.SA', 'MGLU3.SA', 'RENT3.SA', 'PETR4.SA', 'WEGE3.SA', 'ELET6.SA', 'RADL3.SA']

data = yf.download(stocks, start='2021-01-01', end='2022-12-31')['Adj Close']

data

[*********************100%***********************]  7 of 7 completed


,ELET6.SA,ITUB4.SA,MGLU3.SA,PETR4.SA,RADL3.SA,RENT3.SA,WEGE3.SA
Date,,,,,,,
2021-01-04,30.883390,27.964476,25.181740,11.470777,23.486467,65.240555,35.927433
2021-01-05,29.996130,27.783466,24.742058,11.919137,23.420790,64.855530,38.132565
2021-01-06,29.825502,28.597969,23.442999,11.942942,22.876554,61.120907,39.143661
2021-01-07,29.262436,29.711126,23.143217,12.300038,22.585672,64.018135,40.761414
2021-01-08,30.115568,29.702066,23.822723,12.347653,23.242504,68.339912,43.226547
...,...,...,...,...,...,...,...
2022-12-23,42.900005,24.574020,2.680000,20.983215,22.873898,55.563160,38.168091
2022-12-26,42.582867,24.086245,2.660000,20.832857,22.749372,54.043736,37.860764
2022-12-27,41.621845,23.949671,2.520000,20.974861,22.644007,52.027775,37.682316


In [3]:
returns = data.pct_change().dropna()
returns

,ELET6.SA,ITUB4.SA,MGLU3.SA,PETR4.SA,RADL3.SA,RENT3.SA,WEGE3.SA
Date,,,,,,,
2021-01-05,-0.028729,-0.006473,-0.017460,0.039087,-0.002796,-0.005902,0.061378
2021-01-06,-0.005688,0.029316,-0.052504,0.001997,-0.023237,-0.057584,0.026515
2021-01-07,-0.018879,0.038924,-0.012788,0.029900,-0.012715,0.047402,0.041329
2021-01-08,0.029154,-0.000304,0.029361,0.003871,0.029082,0.067509,0.060477
2021-01-11,-0.021813,-0.022547,-0.014681,-0.008355,-0.005248,-0.019437,-0.030630
...,...,...,...,...,...,...,...
2022-12-23,0.015931,0.015726,0.030769,0.047103,0.013152,0.054665,-0.011299
2022-12-26,-0.007393,-0.019849,-0.007463,-0.007166,-0.005444,-0.027346,-0.008052
2022-12-27,-0.022568,-0.005670,-0.052632,0.006816,-0.004632,-0.037302,-0.004713


In [30]:
def portfolio_returns(solution):
    port_returns = 0
    for i in range(len(solution)):
        port_returns += returns.iloc[:, i] * solution[i]
    return port_returns

def cvm(solution):
    """
    Carteira de variância mínima
    """
    port_returns = portfolio_returns(solution)
    return np.std(port_returns)

def constraints(solution):
    bound = (sum(solution) - 1)**2
    return bound

def fitness_function(ga_instance, solution, solution_idx):
    return (cvm(solution) + 100*constraints(solution))


In [31]:
ga = pygad.GA(num_generations=50, num_parents_mating=10,
              fitness_func=fitness_function, sol_per_pop=20,
              gene_type=np.float32, mutation_percent_genes=10,
              gene_space=bounds, num_genes=num_stocks)

ga.run()

c:\Users\Yamac\AppData\Local\Programs\Python\Python310\lib\site-packages\pygad\pygad.py:738: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resutled in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resutled in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")


### Teste 2

In [57]:
def portfolio_generate(df):
    portfolio = df.copy()
    portfolio['Total'] = portfolio.sum()
    portfolio['Total'] = portfolio['Total']  / 100
    portfolio['Total'] = portfolio['Total'] + 1
    portfolio = portfolio[['Total']]
    return portfolio

def portfolio_return(portfolio):
    return np.prod(portfolio['Total']) -1

def fitness_function(ga_instance, solution, solution_idx):
    portfolio = portfolio_generate(returns)
    ret = portfolio_return(portfolio)
    return ret

In [65]:
ga_instance = pygad.GA(num_generations=30, num_parents_mating=50,
                       fitness_func=fitness_function, sol_per_pop=90,
                       init_range_low=0, init_range_high=1,
                       parent_selection_type='sss', keep_parents=30,
                       crossover_type='single_point', mutation_percent_genes=10,
                       random_seed=42, num_genes=7)

ga_instance.run() 

c:\Users\Yamac\AppData\Local\Programs\Python\Python310\lib\site-packages\pygad\pygad.py:738: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resutled in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resutled in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")


In [66]:
ga_instance.best_solution()

(array([6.31302552, 3.92887282, 5.22131094, 5.60241688, 5.04461411,
        5.74429642, 6.69531025]),
 140999.10240085586,
 0)

### Teste 3

In [88]:
teste = [21,32,43,54,65,76,87]

In [89]:
def constraint(solution):
    bounduary_constraint = (sum(solution) - 1)**2
    return bounduary_constraint

def fitness_function(ga_intance, solution, solution_idx):
    fitness = np.dot(teste, solution)
    return fitness -100*constraint(solution)

In [96]:
ga_instance = pygad.GA(num_generations=30, num_parents_mating=50,
                       fitness_func=fitness_function, sol_per_pop=90,
                       init_range_low=0, init_range_high=1,
                       parent_selection_type='sss', keep_parents=30,
                       crossover_type='single_point', mutation_percent_genes=10,
                       random_seed=42, num_genes=7, gene_space=gene_space)

ga_instance.run() 

c:\Users\Yamac\AppData\Local\Programs\Python\Python310\lib\site-packages\pygad\pygad.py:738: UserWarning: The percentage of genes to mutate (mutation_percent_genes=10) resutled in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  warnings.warn(f"The percentage of genes to mutate (mutation_percent_genes={mutation_percent_genes}) resutled in selecting ({mutation_num_genes}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.")


In [99]:
ga_instance.best_solution()

(array([0.03673194, 0.00977085, 0.03142919, 0.01009061, 0.11573761,
        0.14032452, 0.9623102 ]),
 95.50119723987602,
 0)

In [100]:
np.sum([0.03673194, 0.00977085, 0.03142919, 0.01009061, 0.11573761, 0.14032452, 0.9623102])

1.30639492

### Teste 4 - DEAP

In [25]:
n_stocks = 7

In [190]:
creator.create('FitnessMax', base.Fitness, weights=(1.0,-1.0))  # O objetivo é maximizar a solução
creator.create("Individual", list, fitness=creator.FitnessMax)  # Lista com os primeiros indivíduos

c:\Users\Yamac\AppData\Local\Programs\Python\Python310\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\Yamac\AppData\Local\Programs\Python\Python310\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [191]:
def generateES(icls, size, imin, imax):
    ind = icls(random.uniform(imin, imax) for _ in range(size))
    return ind

def calculate_returns(individual):
    x = returns.mul(individual).sum(axis=1)  # Soma dos retornos da carteira dia a dia
    x = x + 1
    x = x.cumprod() - 1
    return x.tail(1)
    
def calculate_risk(individual):
    individual = np.array(individual)
    cov_matrix = returns.cov()
    port_variance = np.dot(individual.T, np.dot(cov_matrix, individual))
    port_risk = np.sqrt(port_variance)
    return port_risk
    
def evaluate_portfolio(individual):
    # Calculate returns and risk based on individual weights
    returns = calculate_returns(individual)
    risk = calculate_risk(individual)

    penalty = abs((np.sum(individual) - 1)**2)  # Penalidade para a soma dos pesos ser 1
    
    # Return a tuple of the fitness values (returns, -risk as it's a maximization problem)
    return float(returns - 100*penalty), float(-risk - 100*penalty)

In [193]:
toolbox = base.Toolbox()
toolbox.register('attr_bool', random.randint, 0, 1)
#toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n_stocks)  # Cria os indivíduos iniciais com n_stocks cromossomos
toolbox.register("individual", generateES, creator.Individual, 7, 0, 1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)  # Coleção dos indívduos iniciais - serão armazenados em uma lista
toolbox.register("evaluate", evaluate_portfolio)
toolbox.register("mate", tools.cxTwoPoint)
#toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.1, indpb=0.1)
toolbox.register("mutate", tools.mutPolynomialBounded, eta=0.5, low=lower_bound, up=upper_bound, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

In [194]:
population_size = 100
num_generations = 50
cx_prob = 0.5
mut_prob = 0.2

In [195]:
population = toolbox.population(n=population_size)
algorithms.eaSimple(population, toolbox, cx_prob, mut_prob, num_generations)

gen	nevals
0  	100   
1  	51    
2  	58    
3  	54    
4  	56    
5  	57    
6  	59    
7  	51    
8  	66    
9  	65    
10 	56    
11 	72    
12 	63    
13 	58    
14 	61    
15 	59    
16 	57    
17 	60    
18 	67    
19 	57    
20 	56    
21 	60    
22 	44    
23 	56    
24 	68    
25 	62    
26 	55    
27 	60    
28 	53    
29 	66    
30 	70    
31 	61    
32 	65    
33 	62    
34 	58    
35 	60    
36 	65    
37 	64    
38 	64    
39 	63    
40 	41    
41 	73    
42 	58    
43 	66    
44 	55    
45 	51    
46 	66    
47 	48    
48 	55    
49 	63    
50 	67    


([[0.41579183817901777,
   0.03241582242137542,
   0.007121255425956985,
   0.06813424819146996,
   0.37255404024520355,
   0.017317211328799598,
   0.07321945296493482],
  [0.41579183817901777,
   0.03241582242137542,
   0.007121255425956985,
   0.06813424819146996,
   0.37255404024520355,
   0.017317211328799598,
   0.07321945296493482],
  [0.41579183817901777,
   0.03241582242137542,
   0.007121255425956985,
   0.06813424819146996,
   0.37255404024520355,
   0.017317211328799598,
   0.07321945296493482],
  [0.41579183817901777,
   0.03241582242137542,
   0.007121255425956985,
   0.06813424819146996,
   0.37255404024520355,
   0.017317211328799598,
   0.07321945296493482],
  [0.41579183817901777,
   0.03241582242137542,
   0.007121255425956985,
   0.06813424819146996,
   0.37255404024520355,
   0.017317211328799598,
   0.07321945296493482],
  [0.41579183817901777,
   0.03241582242137542,
   0.007121255425956985,
   0.06813424819146996,
   0.37255404024520355,
   0.017317211328799598,

In [196]:
best_individual = tools.selBest(population, k=1)[0]

In [197]:
np.sum(best_individual)

0.9988438089344784

In [206]:
best_individual

[0.4344885054516099,
 0.03241582242137542,
 0.007121255425956985,
 0.06813424819146996,
 0.37255404024520355,
 0.01091048423392782,
 0.07321945296493482]

In [210]:
data.head(1)

,ELET6.SA,ITUB4.SA,MGLU3.SA,PETR4.SA,RADL3.SA,RENT3.SA,WEGE3.SA
Date,,,,,,,
2021-01-04,30.88339,27.964476,25.18174,11.470777,23.486467,65.240555,35.927433


In [212]:
print('Retorno ELET6:', data['ELET6.SA'].iloc[-1] / data['ELET6.SA'].iloc[0] - 1)
print('Retorno ITUB4:', data['ITUB4.SA'].iloc[-1] / data['ITUB4.SA'].iloc[0] - 1)
print('Retorno MGLU3:', data['MGLU3.SA'].iloc[-1] / data['MGLU3.SA'].iloc[0] - 1)
print('Retorno PETR4:', data['PETR4.SA'].iloc[-1] / data['PETR4.SA'].iloc[0] - 1)
print('Retorno RADL3:', data['RADL3.SA'].iloc[-1] / data['RADL3.SA'].iloc[0] - 1)
print('Retorno RENT3:', data['RENT3.SA'].iloc[-1] / data['RENT3.SA'].iloc[0] - 1)
print('Retorno WEGE3:', data['WEGE3.SA'].iloc[-1] / data['WEGE3.SA'].iloc[0] - 1)

Retorno ELET6: 0.3452203033006047
Retorno ITUB4: -0.12786978140413097
Retorno MGLU3: -0.8911909967094072
Retorno PETR4: 0.7841265296311992
Retorno RADL3: -0.03260733007933536
Retorno RENT3: -0.19019425321094907
Retorno WEGE3: 0.06264216549281754


In [213]:
np.dot([0.3452203033006047, -0.12786978140413097, -0.8911909967094072, 0.7841265296311992, -0.03260733007933536, -0.19019425321094907, 0.06264216549281754], best_individual)

0.183292243494318

In [204]:
optimized_portfolio = returns.mul(best_individual).sum(axis=1)  # Soma dos retornos da carteira dia a dia
optimized_portfolio = optimized_portfolio + 1
optimized_portfolio = optimized_portfolio.cumprod() - 1
px.line(optimized_portfolio, title='Estratégia otimizada', x=optimized_portfolio.index, y=optimized_portfolio.values)

In [215]:
returns

,ELET6.SA,ITUB4.SA,MGLU3.SA,PETR4.SA,RADL3.SA,RENT3.SA,WEGE3.SA
Date,,,,,,,
2021-01-05,-0.028729,-0.006473,-0.017460,0.039087,-0.002796,-0.005902,0.061378
2021-01-06,-0.005688,0.029316,-0.052504,0.001997,-0.023237,-0.057584,0.026515
2021-01-07,-0.018879,0.038924,-0.012788,0.029900,-0.012715,0.047402,0.041329
2021-01-08,0.029154,-0.000304,0.029361,0.003871,0.029082,0.067509,0.060477
2021-01-11,-0.021813,-0.022547,-0.014681,-0.008355,-0.005248,-0.019437,-0.030630
...,...,...,...,...,...,...,...
2022-12-23,0.015931,0.015726,0.030769,0.047103,0.013152,0.054665,-0.011299
2022-12-26,-0.007393,-0.019849,-0.007463,-0.007166,-0.005444,-0.027346,-0.008052
2022-12-27,-0.022568,-0.005670,-0.052632,0.006816,-0.004632,-0.037302,-0.004713


In [214]:
vanilla_portfolio = returns.sum(axis=1)
vanilla_portfolio

Date
2021-01-05    0.039104
2021-01-06   -0.081185
2021-01-07    0.113173
2021-01-08    0.219150
2021-01-11   -0.122712
                ...   
2022-12-23    0.166047
2022-12-26   -0.082712
2022-12-27   -0.120701
2022-12-28    0.148483
2022-12-29   -0.037349
Length: 496, dtype: float64

In [205]:
vanilla_portfolio = returns.sum(axis=1)
vanilla_portfolio = vanilla_portfolio + 1
vanilla_portfolio = vanilla_portfolio.cumprod() - 1
px.line(vanilla_portfolio, title='Estratégia sem otimização', x=vanilla_portfolio.index, y=vanilla_portfolio.values)